<a href="https://colab.research.google.com/github/GimenesPaula/GimenesPaula/blob/main/Projeto%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazer Upload planilhas
1.   Launches
2.   Inci
3.   Sales
4.   Project

In [25]:
#Launches
from google.colab import files
uploaded = files.upload()

Saving 2023.xlsx to 2023.xlsx


In [2]:
filename = next(iter(uploaded))

In [3]:
# INCI name produtos
from google.colab import files
inci = files.upload()

Saving inci (1).xlsx to inci (1).xlsx


In [4]:
filename2 = next(iter(inci))

In [223]:
#Projetos Distribuidores
from google.colab import files
proj = files.upload()

Saving Projetos.xlsx to Projetos.xlsx


In [6]:
filename3 = next(iter(proj))

In [7]:
#Vendas Distribuidores
from google.colab import files
dist = files.upload()

Saving Vendas.xlsx to Vendas.xlsx


In [8]:
filename4 = next(iter(dist))

# Bibliotecas Phyton

In [224]:
#Carrega Bibliotecas
import pandas as pd
import numpy as np
import re

In [225]:
pip install unidecode

In [226]:
from unidecode import unidecode

##Carrega tabela do Mintel

In [227]:
#Carrega o banco de dados como tabela
data = '/content/'+filename
df_launches = pd.read_excel(data)
pd.options.display.max_columns = None
df_launches.nunique()

Record ID                       5334
Product                         3231
Brand                           3284
Company                          982
Market                            23
Date Published                   228
Category                           4
Sub-Category                      75
Price in local currency         1465
Ingredients (On pack)           5209
Free from                        107
Manufacturer                     747
Location of Manufacture           41
Manufacturer Company Address     633
dtype: int64

##Carrega Tabela de INCI

In [228]:
data = '/content/'+filename2
df_inci = pd.read_excel(data)
pd.options.display.max_columns=None
df_inci.nunique()

Produto        52
Ingrediente    48
Prioridade      3
dtype: int64

##Carrega Tabela de Projetos

In [229]:
data = '/content/'+filename3
df_proj = pd.read_excel(data)
pd.options.display.max_columns=None
df_proj.nunique()

Distributor      4
Customer       628
UF              21
Material       201
dtype: int64

## Carrega Planilha de Vendas do Distribuidor

In [230]:
data = '/content/'+filename4
df_dist = pd.read_excel(data)
pd.options.display.max_columns=None
df_dist.nunique()

Customer(at distrib)    1819
Country, Customer          1
Distributor                3
Material                 130
Year                     404
dtype: int64

# Define funções

In [231]:
#this checks if any combination of INCI as present in Ingredient
def verifica_ingrediente(formula,produto,material):
  quantidade = len(produto.difference(formula))
  if quantidade == 0:
    return material
  return None

In [232]:
#If last code is true, this returns the material name
def procura_produtos(formula, produtos, materiais):
  formula = formula.copy()
  resultados = []
  for prod, mat in zip(produtos, materiais):
    resultado = verifica_ingrediente(formula, prod, mat)
    if resultado is not None:
      formula = formula.difference(prod) ## Para remover os ingredientes já encontrados numa nova busca.
      resultados.append(resultado)
  return resultados

In [233]:
#this code transpose data. Used when we bring each category and the number of lauches.
def transpor (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:[])

In [234]:
#Code to solve an issue from algoritms
def to_set(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set)
        else:
            return set(x.sum())
    except:
        pass

In [235]:
#ode to solve an issue from algoritms
def to_set_2(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set_2)
        else:
            return list(set(x))
    except:
        pass

In [236]:
#This is a dictionary of words that may be removed from Company, Manufacture
#in order to group manufacture despite of spelling mistakes
def formatar_fabricantes(coluna):
  remover = [
             'fabrica', 'fabricacao', 'manuf', 'tercerizacao', 'manufacturing',
             'farmacia', 'farmacias', 'farmaceuticos', 'drogaria','drogarias', 'farm',
             'cosmetico', 'cometico','cosmeticos', 'cometicos', 'cosmetics',
             'cosmiatria', 'cosmetica', 'cosmeticas', 'cosm', 'cosme', 'kosmetics',
             'ltda','epp', 'eireli', 'me', 'eirell', 'sa', 'eir', 'ltd',
             'indústria','industria', 'industrias', 'industrial', 'indutrial', 'ind', 'industriae',
             'comercio', 'comecio', 'comercial', 'corporation', 'com',
             'imp', 'exp',  'prod', 'exportacao','importacao','importadora', 'import',
             'do', 'e', 'para', 'de', 'le', 'la',
             'do brasil', 'aerossois', 'aerosol', 'aerossol', 'tecnologia',
             'laboratorio','laboratorios', 'laboratories', 'laboratoire', 'instituto', 'group',
             'pesquisa', 'desenvolvimento', 'naturais', 'professional', 'aromatizantes',
             'produtos', 'Produto', 'saneantes', 'quimica', 'quimicos', 'quimico', 'farmaceutica', 'veterinário',
             'higiene','pessoal', 'limpeza', 'beleza', 'botanico',
             'perfumes','fragrancias', 'perfumaria', 'medicamentos', 'perf',
             'distribuidora', 'dist',
             'international', 'internacional', 'sulamericana', 'nacional',
             'nbsp;', 'veer','serv', 'vajerista',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .apply(unidecode)
    .str.lower()
    .replace('\.', ' ',regex=True)
    .replace('\/', '',regex=True)
    .replace('\-', '',regex=True)
    .replace(pat,'',regex=True)
    .replace(',', '',regex=True)
    .replace('\s+','',regex=True) #Tirar numeros
    .str.strip()
  )
  return coluna

In [237]:
#Dis dict clean the name of material
def formatar_material(coluna):
  remover = ['BELSIL®', 'BELSIL', 'BELSIM',
             'DRUM', 'ST', 'PE', 'CANISTER', 'RD', 'CARTON', 'IBC PE', '2', 'WOPAL',
             'SBPAL', 'STPAL', 'HOBBOCK', 'SACK', 'BOTTLE',
             '0,45KG','10KG','15KG','25KG', '30G', '60G', '100KG', '140KG', '150KG', '160KG',
             '180KG', '190KG', '200KG', '210KG', '900KG', '950KG', '190',
             'KG', 'AMOSTRA', 'PA', 'XY', 'CP', 'SILICONE ELASTOMER GEL',
             'BR', 'DE', ' E', 'BLOCKED',
             'EMB. 0,100 KG', 'Ver  Pf 100', 'USP',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .replace(pat,'',regex=True)
    .replace('\#','', regex=True)
    .replace('\"','', regex=True)
    .str.replace('\/', '',regex=True)
    .str.replace('\-', '',regex=True)
    .str.replace('\*', '',regex=True)
    .str.title()
    .str.strip()
  )
  return coluna

# Mintel

## Define Estados


In [238]:
dict = {'AC':'AC', 'AL':'AL', 'AM':'AM', 'AP':'AP', 'BA':'BA', 'CE':'CE', 'DF':'DF', 'ES':'ES', 'GO':'GO', 'MA':'MA',
     'MG':'MG', 'MS':'MS', 'MT':'MT', 'PA':'PA', 'PB':'PB', 'PE':'PE', 'PI':'PI', 'PR':'PR', 'RJ':'RJ', 'RN':'RN',
     'RO':'RO', 'RR':'RR', 'RS':'RS', 'SC':'SC', 'SE':'SE', 'SP':'SP', 'TO':'TO', 'Sao Paulo':'SP', 'São Paulo':'SP',
     'Sao Paolo':'SP','Osasco': 'SP', 'Mogi das Cruzes': 'SP', 'Cotia':'SP', 'Diadema':'SP', 'São Roque':'SP',
     'São Sebastião':'SP', 'Lins':'SP', 'Cajamar':'SP', 'Pirassununga':'SP', 'Paulista': 'SP', 'São Paolo':'SP',
     'Elisario':'SP', 'Valinhos':'SP', 'Piracaia':'SP', 'São José do Rio Preto':'SP', 'Sao José do Rio Preto':'SP',
     'São José dos Campos':'SP',      'Araras':'SP', 'São Bernardo do Campo':'SP', 'São José dos Campos':'SP',
     'Sao Caetano do Sul':'SP', 'São José do Rio Preto':'SP', 'Brodowski':'SP', 'Cajuru':'SP', 'Itaquaquecetuba':'SP',
     'Itupeva':'SP', 'SO':'SP', 'Mairiporã':'SP', 'Itaquera':'SP', 'Franco da Rocha':'SP', 'Guarulhos':'SP',
     'Bairro Jardim America':'SP', 'Jardim Brazil':'SP', 'Agudos':'SP', 'Itapira':'SP', 'Rib. Preto':'SP',
     'Rio de Janeiro':'RJ', 'Rio De Janeiro':'RJ', 'Nova Iguaçu': 'RJ', 'Nova Iguacu': 'RJ', 'Nivopolis':'RJ',
     'Duque de Caixas':'RJ', 'Engenheiro Pedreira':'RJ',
     'Vila Velha':'ES', 'Marechal Floriano':'ES',
     'Belo Horizonte':'MG', 'Minas Gerais':'MG', 'Pocos de Caldas':'MG', 'Viçosa':'MG', 'Vicosa':'MG',
     'Pouso Alegre': 'MG', 'Uberlândia':'MG', 'Uberlandia':'MG', 'Ipatinga':'MG', 'Mirabela':'MG','Nova Lima':'MG',
     'São José da Lapa':'MG', 'Sao José da Lapa':'MG', 'Sao Jose da Lapa':'MG',
     'Porto Alegre':'RS', 'Estância Velha': 'RS', 'Caxias do Sul':'RS', 'Passo Fundo':'RS', 'Gravatai':'RS',
     'Santa Catarina':'SC', 'Joinville':'SC','Camboriu':'SC', 'Maluche':'SC', 'Araquari':'SC', 'Criciúma': 'SC',
     'Criciuma': 'SC',
     'Colombo':'PR', 'Paraná':'PR', 'Curitiba':'PR', 'Sao José dos Pinhais':'PR','BR 116 Km 85':'PR', 'Cambé':'PR',
     'Goiana':'GO', 'Goiânia':'GO', 'Anápolis':'GO', 'Anapolis':'GO', 'Brasilia': 'DF', 'Go':'GO',
     'Rondônia':'RO','Novo Hamburgo':'RO','Manaus':'AM', 'Pará':'PA',
     'Bahia':'BA', 'Lauro de Freites': 'BA', 'Av. Queira Deus':'BA', 'Vila Cia Sul':'BA',
     'Recife':'PE', 'Camaragibe': 'PE', 'Fortaleza':'CE', 'Ceará':'CE'}

In [239]:
df_launches['State'] = df_launches['Manufacturer Company Address'].str.extract('('+'|'.join(dict)+')',expand=False).map(dict)
#Exibe a quantidade de casos únicos
df_launches['State'].value_counts(dropna=False)
#Em output se filtra estados vazios

NaN    3582
SP     1091
MG      138
RJ      122
PR       98
GO       95
RS       71
SC       39
BA       34
CE       26
PE       14
ES       11
DF        4
PI        3
AL        2
SE        2
AM        1
PA        1
Name: State, dtype: int64

## Search dictOTHERS words in the ingredient, in order to identify cases where the words occure.

In [240]:
#Dicionário Silicones geral
dictOTHERS = {'methicone':'1', 'Dimethicone':'1','methicone Crosspolymer':'1','methiconol':'1',
              'ylsiloxysilicate':'1', 'ylsilsesquioxane':'1', 'Disiloxane':'1', 'Silica':'1','siloxane':'1'}

In [241]:
df_launches['Silicone'] = df_launches['Ingredients (On pack)'].str.extract('('+'|'.join(dictOTHERS)+')',expand=False).map(dictOTHERS)

##Search "Free From"


In [242]:
dictSil = {'Silicone':'1'}

In [243]:
df_launches['Free from'] = df_launches['Free from'].str.extract('('+'|'.join(dictSil)+')',expand=False).map(dictSil)

## Search ingredients and identify products that mache INCI



In [244]:
df_inci.dropna(inplace=True)
df_inci['Produto'] = formatar_material(df_inci['Produto'])

In [245]:
#Cria uma lista iterável dos ingredientes nos Produtos
df_inci['Ing'] = formatar_material(df_inci['Ingrediente']).str.split(', ').apply(set)

In [246]:
#Conta a quantidade de ingredientes
df_inci.sort_values('Prioridade', ascending=True, inplace=True)

In [247]:
 #Dicionário de palavras a remover da coluna Ingredientes no Mintel
 dictIng={' Andor ':',',
          ', ':',',
          ' ,':',',
          ' (And) ':',',
          }

In [248]:
#Cria uma lista iterável das formulas do mintel
df_launches['Ingredients (On pack)'] = df_launches['Ingredients (On pack)'].astype(str)
df_launches['Ing'] = formatar_material(df_launches['Ingredients (On pack)']).replace(dictIng,regex=True).dropna().str.split(',').apply(set)

In [249]:
#Aplica a função Procura Produtos
df_launches[''] = df_launches['Ing'].apply(lambda formulacao:procura_produtos(formulacao, df_inci['Ing'], df_inci['Produto']))

## Remover Clientes  da planilha Mintel


In [287]:
dictClientes = {'Devintex Cosméticos':'1', 'Devintex':'1','Procosa Produtos de Beleza':'1', 'Colgate-Palmolive':'1', 'Unilever':'1',
                'Savoy Indústria de Cosméticos':'1', 'Beiersdorf':'1', 'Johnson & Johnson':'1',
                'Procter & Gamble':'1','Química Amparo':'1', "Oréal":'1', 'Procosa':'1',
                'Unilever Brasil Industrial':'1', 'Botica Comercial Farmacêutica':'1'}
dictMarcas = { 'Boticário':'1', 'Siàge':'1', 'Natura ':'1', 'Avon ':'1', 'Eudora':'1',
              'Wella':'1', 'The Body Shop':'1', 'Quem Disse,':'1', 'Salon Line':'1', 'Maybelline':'1'}
df_launches['Clientes']= df_launches['Manufacturer'].str.extract('('+'|'.join(dictClientes)+')',expand=False).map(dictClientes)
df_launches['Clientes2'] = df_launches['Brand'].str.extract('('+'|'.join(dictMarcas)+')',expand=False).map(dictMarcas)
df_ = df_launches[df_launches['Clientes']=='1']
df_ = df_[df_['Clientes2']=='1']
df_.to_excel("LancClientes.xlsx")

In [286]:
#Remove clientes  da base de tratamento
df_launches = df_launches[df_launches['Clientes']!='1']
df_launches = df_launches[df_launches['Clientes2']!='1']

## Cria Indice para junção

In [252]:
df_launches.set_index(['Record ID'])
df_launches['Date Published'] = pd.to_datetime(df_launches['Date Published'], format='%Y-%m-%d')
df_launches['Quarter'] = df_launches['Date Published'].dt.quarter
df_launches['Year'] = df_launches['Date Published'].dt.year

<ipython-input-252-83b2b95c05d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_launches['Date Published'] = pd.to_datetime(df_launches['Date Published'], format='%Y-%m-%d')
<ipython-input-252-83b2b95c05d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_launches['Quarter'] = df_launches['Date Published'].dt.quarter
<ipython-input-252-83b2b95c05d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [253]:
#Cria a coluna que irá indexar a mistura com a planilha do mintel
df_launches['Manufacturer']=df_launches['Manufacturer'].astype(str)
df_launches['KeyManuf'] = formatar_fabricantes(df_launches['Manufacturer'])

<ipython-input-253-c129d5709b06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_launches['Manufacturer']=df_launches['Manufacturer'].astype(str)
<ipython-input-253-c129d5709b06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_launches['KeyManuf'] = formatar_fabricantes(df_launches['Manufacturer'])


## Report: Lauches with Identified Material

In [254]:
df_launches.columns

Index(['Record ID', 'Product', 'Brand', 'Company', 'Market', 'Date Published',
       'Category', 'Sub-Category', 'Price in local currency',
       'Ingredients (On pack)', 'Free from', 'Manufacturer',
       'Location of Manufacture', 'Manufacturer Company Address', 'State',
       'Silicone', 'Ing', '', 'Clientes', 'Clientes2', 'Quarter', 'Year',
       'KeyManuf'],
      dtype='object')

In [255]:
col=['Quarter','Sub-Category', 'Product', 'Brand', 'Company', 'Manufacturer', 'State', 'Free from', 'Price in local currency',
     'Silicone', '', 'Ing', 'Manufacturer Company Address',]

In [256]:
df_launches[col].to_excel("Lancamentos Linha a Linha.xlsx")

## Generate a Reports on Manufacture activities

In [257]:
transpor(df_launches, 'Category', '')

In [258]:
df_launches.rename(columns={'Skincare':'Skin', 'Hair Products':'Hair',
                          'Soap & Bath Products':'Bath', 'Record ID': 'Total Lançamentos'},
                 inplace=True)

In [259]:
df_a = df_launches.groupby(['KeyManuf', 'State'])[['Hair', 'Skin', 'Colour Cosmetics', 'Bath']].apply(to_set)

In [260]:
df_b = df_launches.groupby(['KeyManuf', 'State']).agg({
                                            'Total Lançamentos': 'count',
                                             'Silicone':'count',
                                             'Free from':'count',
                                             })

In [261]:
df_launches_fab = pd.concat([df_a, df_b], axis=1).reset_index()

In [262]:
df_launches_fab.to_excel('Manufacturer x Project Status.xlsx')

In [263]:
df_b.to_excel('Manufacture Lauches.xlsx')

# Prepare Sales Report

In [264]:
#Garantir que apenas serão considerados os clientes com vendas
df_dist['Year'].dropna(inplace=True)

In [265]:
df_dist['Material']= formatar_material(df_dist['Material'])
df_dist['Material'] = df_dist['Material'].astype(str)

In [266]:
df_dist['KeyManuf'] = formatar_fabricantes(df_dist['Customer(at distrib)'])

In [267]:
#agrupo os produtos em uma lista, por cliente
df_dist_unic = df_dist.groupby(['KeyManuf','Distributor']).agg({'Year':'sum','Material':lambda x:list(set(x))}).reset_index()

In [268]:
df_dist_unic['Material'] = df_dist_unic['Material'].apply(set)
df_dist_unic.rename(columns={'Material': 'Sales'}, inplace=True)

In [269]:
df_dist_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)

# Prepare Project Report

In [271]:
df_proj['Material']= formatar_material(df_proj['Material'])

In [272]:
df_proj['Customer'] = df_proj['Customer'].astype(str)

In [273]:
df_proj['KeyManuf'] = formatar_fabricantes(df_proj['Customer'])

In [274]:
#agrupo os produtos em uma lista, por cliente
df_proj_unic = df_proj.groupby(['KeyManuf','Distributor']).agg({'Material':lambda x:list(set(x))}).reset_index()

In [275]:
df_proj_unic['Material'] = df_proj_unic['Material'].apply(set)
df_proj_unic.rename(columns={'Material': 'Project'}, inplace=True)

In [276]:
df_proj_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)

# Group report Sales and Project

In [280]:
df_proj_vend = df_proj_unic.merge(df_dist_unic, on=['KeyManuf', 'Distributor'], how='left')
df_vend_proj = df_dist_unic.merge(df_proj_unic, on=['KeyManuf', 'Distributor'], how='left')

In [282]:
df_proj_vend.to_excel('Projetos com Vendas.xlsx')
df_vend_proj.to_excel('Vendas com Projetos.xlsx')

# Group all reports

In [283]:
df_launches_vend_proj = df_launches_fab.merge(df_vend_proj, on='KeyManuf', how='left')

In [284]:
df_launches_vend_proj.rename(columns={'': ' Prod Poss','Customer(at distrib)':'Customer Sales','Distributor':'Dist. Sales',
                                    'Sales': 'Prod Sales','Year': 'Sales Volume', 'Customer': 'Customer Proj Fab',
                                    'Distributor':'Dist Proj Fab', 'Project':'Prod Proj Fab',
                                    'Fase':'Fase Proj Fab',
                                         },
                                inplace=True)

,KeyManuf,State,Hair,Skin,Colour Cosmetics,Bath,Total Lançamentos,Silicone,Free from,Dist Proj Fab,Sales Volume,Prod Sales,Material
390,vita&aroma,SP,NaN,NaN,NaN,NaN,1,0,0,FOCUS QUIMICA,4.76,"{Gb 3021, Gb 150}",NaN
106,cosinter,PR,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN


# Generate a Report will all information

In [285]:
df_launches_vend_proj.to_excel('Final.xlsx')